# Passive suicidality in a repressive U.S. political context: Aim I

_WIP - NOT FOR DISTRIBUTION_

> aim_i_train_tune_predict.ipynb<br>
> Simone J. Skeen (10-07-24)

1. [Prepare](xx)
2. [Write](xx)
3. [Preprocess](xx)
4. [Train-Adapt-Test](xx)
5. [Tune](xx)
6. [Infer](xx)
7. [Explain](xx)
Interrogate
Calibrate

### 1. Prepare
Installs, imports, and downloads requisite models and packages. Organizes RAP-consistent directory structure.
***

**Install**

In [ ]:
%%capture

%pip install accelerate
%pip install bitsandbytes
%pip install causalnlp
%pip install contractions
%pip install datasets
%pip install evaluate
%pip install lime
%pip install peft
#%pip install trl
%pip install unidecode
%pip install wandb

#%pip uninstall -y pyarrow datasets
#%pip install pyarrow datasets

#!python -m spacy download en_core_web_lg --user

**Import**

In [ ]:
import accelerate
import ast
import bitsandbytes as bnb
import contractions
#import en_core_web_lg
import gzip
import huggingface_hub
import json
import lime
import logging
import matplotlib.pyplot as plt
import nltk
import numpy as np
import os
import pandas as pd
import peft
import random
import re
import seaborn as sns
import sklearn
import spacy
import string
import torch
#import trl
import wandb.sdk
import warnings

from causalnlp import Autocoder, CausalInferenceModel
from google.colab import drive
from lightgbm import LGBMClassifier
from lime.lime_text import LimeTextExplainer
from nltk.text import Text
from sklearn.feature_extraction import text
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import (
                                     KFold,
                                     ParameterGrid,
                                     StratifiedKFold,
                                     train_test_split,
                                     )
from sklearn.metrics import (
                             average_precision_score,
                             classification_report,
                             f1_score,
                             matthews_corrcoef,
                             )
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.utils.multiclass import type_of_target
from textblob import TextBlob
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss
from torch.utils.data import (
                              DataLoader,
                              Dataset,
                              TensorDataset,
                              )
from tqdm import tqdm
from transformers import (
                          AdamW,
                          BertForSequenceClassification,
                          BertTokenizer,
                          DataCollatorForLanguageModeling,
                          DistilBertForSequenceClassification,
                          DistilBertTokenizer,
                          RobertaForSequenceClassification,
                          RobertaTokenizer,
                          Trainer,
                          TrainingArguments,
                          )
from unidecode import unidecode

#spacy.cli.download('en_core_web_lg')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

pd.options.mode.copy_on_write = True

pd.set_option(
              'display.max_columns',
              None,
              )

pd.set_option(
              'display.max_rows',
              None,
              )

warnings.simplefilter(
                      action = 'ignore',
                      category = FutureWarning,
                      )

#!python -m prodigy stats

**Mount gdrive**

In [ ]:
drive.mount(
            '/content/drive',
            #force_remount = True,
            )

**Structure directories**

In [ ]:
%cd /content/drive/My Drive/Colab/bar_policy_suicidality
#%cd /content/drive/My Drive/#<my_project_folder>

#%mkdir bar_policy_suicidality
#%cd bar_policy_suicidality

In [ ]:
#%mkdir inputs outputs code temp

In [ ]:
#%cd inputs
#%mkdir annotation archives data

In [ ]:
#%cd ../outputs
#%mkdir models tables figures

In [ ]:
bar_policy_suicidality/
├── inputs/
│   ├── archives
│   │   └── ### archive name TKTK
│   └── data
│       └── d_annotated.xlsx
├── outputs/
│   ├── models
│   ├── tables
│   └── figures
├── code/
└── temp/

### 2. Write
Writes and imports preprocess.py, train.py, redact.py, predict.py.
***

In [ ]:
%cd code

#### preprocess.py

**_augment_training_data_with_rationales_**

In [ ]:
%%writefile preprocess.py

import pandas as pd

def augment_training_data_with_rationales(df):
    """
    Identifies all pos_1 strn, duplicates as new row below, replaces new row 'text' with appended concatenated asp-dep-val 'rtnl'.
    """
    augmented_rows = []

    for index, row in df.iterrows():
        augmented_rows.append(row)

        if row['strn'] == 1:
            duplicate_row = row.copy()
            duplicate_row['text'] = duplicate_row['rtnl']
            augmented_rows.append(duplicate_row)

    df_augmented = pd.DataFrame(augmented_rows)

    return df_augmented

**_dummy_code_augmented_rows_**

In [ ]:
%%writefile -a preprocess.py

import pandas as pd

def dummy_code_augmented_rows(df):
    """
    Identifies all rationale-augmented rows in df, dummy codes for deletion prior to evaluation.
    """
    df = df.reset_index(drop = True)

    df['aug'] = 0

    for i in range(1, len(df)):
        if df.at[i, 'rtnl'] != '.' and df.at[i, 'rtnl'] == df.at[i-1, 'rtnl']:
            df.at[i, 'aug'] = 1

    return df

**_read_and_append_jsonl_archives_**

In [ ]:
%%writefile -a preprocess.py

import os
import pandas as pd

def read_and_append_jsonl_archives(directory, chunk_size = 10000):
    """
    Reads and appends JSONL archives from Arctic Shift archives dir.
    """
    d_posts = pd.DataFrame()
    d_comments = pd.DataFrame()

    for filename in os.listdir(directory):
        if filename.endswith("_posts.jsonl"):
            filepath = os.path.join(
                                    directory,
                                    filename,
                                    )
            for chunk in pd.read_json(
                                      filepath,
                                      lines = True,
                                      chunksize = chunk_size,
                                      ):
                d_posts = pd.concat([d_posts,
                                     chunk], ignore_index = True,
                                    )

        elif filename.endswith("_comments.jsonl"):
            filepath = os.path.join(
                                    directory,
                                    filename,
                                    )
            for chunk in pd.read_json(
                                      filepath,
                                      lines = True,
                                      chunksize = chunk_size,
                                      ):
                d_comments = pd.concat([d_comments,
                                        chunk], ignore_index = True,
                                       )

    return d_posts, d_comments

#### train.py

**_set_seed_**

In [ ]:
%%writefile train.py

import torch
import numpy as np
import random

def set_seed(seed):
    """
    Set random seeds for reproducibility in Pytorch.
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

**_train_eval_save_bl_models_**

In [ ]:
%%writefile -a train.py

import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import (
                          BertForSequenceClassification,
                          RobertaForSequenceClassification,
                          DistilBertForSequenceClassification,
                          BertTokenizer,
                          RobertaTokenizer,
                          DistilBertTokenizer,
                          get_linear_schedule_with_warmup,
                          )
from sklearn.metrics import f1_score, matthews_corrcoef, average_precision_score
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.nn import CrossEntropyLoss
import numpy as np
import pandas as pd

def train_eval_save_bl_models(target_datasets, targets_and_class_weights, models, save_path, cycle, hyperparameter_grid):
    """
    Fine-tune and eval pre-trained baseline LMs on multiple targets using target-specific train and test datasets.

    Training sets d_train_{target} are split using 5-fold cross-validation: 4 folds for training and 1 fold for validation. Training
    folds use augmented data; validation folds use original data. The best model is selected based on average performance across the
    folds and evaluated on separate test sets d_test_{target}.

    Parameters:
    -----------
    target_datasets : dict
        A dictionary where keys are target names and values are tuples containing the train and test datasets for each target.
        For example: {'asp': (d_train_asp, d_test_asp), 'dep': (d_train_dep, d_test_dep)}

    targets_and_class_weights : dict
        A dictionary where keys are target names and values are lists of class weights corresponding to each target.

    models : dict
        A dictionary of models to evaluate. Each key is a model name, and each value is a tuple containing:
        - the model class,
        - the tokenizer class,
        - the name of the pre-trained model.

    save_path : str, optional
        The path where the best models will be saved.

    cycle : str
        The cycle identifier: 'benchmark', indicating performance with default params; 'adapted', indicating performance
        with in-domain adapted params.

    hyperparameter_grid : dict
        Dictionary containing hyperparameter values: batch_size, gradient_accumulation_steps, learning_rate, num_epochs, warmup_steps, and weight_decay.

    Returns:
    --------
    d_{cycle}_performance : pd.DataFrame
        A df containing performance metrics for each target and model per fold per pre-specified cycle,
        and final evaluation on test data.
    """

    # verify cycle

    print(f"CYCLE: {cycle}")

    # check CUDA

    print("CUDA: ", torch.cuda.is_available())
    use_cuda = torch.cuda.is_available()

    # set seed

    set_seed(56)

    # unpack hyperparameters

    batch_size = hyperparameter_grid.get('batch_size', 4)
    gradient_accumulation_steps = hyperparameter_grid.get('gradient_accumulation_steps', 1)
    learning_rate = hyperparameter_grid.get('learning_rate', 2e-5)
    num_epochs = hyperparameter_grid.get('num_epochs', 2)
    warmup_steps = hyperparameter_grid.get('warmup_steps', 0)
    weight_decay = hyperparameter_grid.get('weight_decay', 0.0)

    # best target x model F1 tracking

    best_f1_scores = {target: {'score': 0, 'model': None, 'model_instance': None} for target in targets_and_class_weights}
    results = []

    # training loop: target x model

    for target, class_weights in targets_and_class_weights.items():
        print("\n======================================================================================")
        print(f"Label: {target}")
        print("======================================================================================")

        # target-specific datasets

        d_train, d_test = target_datasets[target]

        # split augmented v. non-augmented data

        d_train_aug = d_train[d_train['aug'] == 1]
        d_train_no_aug = d_train[d_train['aug'] == 0]

        # prepare fold-wise training v. validation data

        X_train_aug, y_train_aug = d_train_aug['text'].values, d_train_aug[target].values
        X_train_no_aug, y_train_no_aug = d_train_no_aug['text'].values, d_train_no_aug[target].values
        X_test, y_test = d_test['text'].values, d_test[target].values

        # determine target type, encode (as needed)

        target_type = 'binary' if len(np.unique(y_train_aug)) <= 2 else 'multiclass'
        le = LabelEncoder() # Using separate LabelEncoder for each target data group to avoid encoding mismatch issues.
        if target_type == 'binary':
            #le = LabelEncoder()
            y_train_aug = le.fit_transform(y_train_aug)
            y_train_no_aug = le.fit_transform(y_train_no_aug) # Re-encode with new encoder
            y_test = le.transform(y_test)

        # define k folds

        k_fold = StratifiedKFold(
                                 n_splits = 5,
                                 shuffle = True,
                                 random_state = 56,
                                 )

        for model_name, (model_class, tokenizer_class, pretrained_model_name) in models.items():
            print(f"\nFine-tuning {model_name} for {target}")
            print("--------------------------------------------------------------------------------------")

            fold_f1, fold_mcc, fold_auprc = [], [], []  ### store fold-wise performance metrics

            # initialize tokenizer

            tokenizer = tokenizer_class.from_pretrained(pretrained_model_name)

            for fold_idx, (train_no_aug_idx, valid_idx) in enumerate(k_fold.split(X_train_no_aug, y_train_no_aug)):
                print(f"\nFold {fold_idx + 1}/5")

                # create training set: combine aug = 1 (augmented) with fold-specific aug = 0 (non-augmented)

                X_train_fold_aug = X_train_aug
                y_train_fold_aug = y_train_aug

                X_train_fold_no_aug, X_valid_fold = X_train_no_aug[train_no_aug_idx], X_train_no_aug[valid_idx]
                y_train_fold_no_aug, y_valid_fold = y_train_no_aug[train_no_aug_idx], y_train_no_aug[valid_idx]

                # combine augmented and non-augmented training data

                X_train_fold = np.concatenate([X_train_fold_aug, X_train_fold_no_aug])
                y_train_fold = np.concatenate([y_train_fold_aug, y_train_fold_no_aug])

                # tokenize training and validation data

                encoded_train = tokenizer(
                                          X_train_fold.tolist(),
                                          padding = True,
                                          truncation = True,
                                          return_tensors = 'pt',
                                          )

                encoded_valid = tokenizer(
                                          X_valid_fold.tolist(),
                                          padding = True,
                                          truncation = True,
                                          return_tensors = 'pt',
                                          )

                train_dataset = TensorDataset(
                                              encoded_train['input_ids'],
                                              encoded_train['attention_mask'],
                                              torch.tensor(y_train_fold),
                                              )

                valid_dataset = TensorDataset(
                                              encoded_valid['input_ids'],
                                              encoded_valid['attention_mask'],
                                              torch.tensor(y_valid_fold),
                                              )

                train_loader = DataLoader(
                                          train_dataset,
                                          batch_size = batch_size,
                                          shuffle = True,
                                          )

                valid_loader = DataLoader(
                                          valid_dataset,
                                          batch_size = batch_size,
                                          shuffle = False,
                                          )

                # instantiate model

                model = model_class.from_pretrained(pretrained_model_name)

                # migrate to CUDA

                use_cuda = torch.cuda.is_available()
                if use_cuda:
                    model = model.cuda()

                # set optimizer + scheduler

                optimizer = torch.optim.AdamW(
                                              model.parameters(),
                                              lr = learning_rate,
                                              weight_decay = weight_decay,
                                              )

                total_steps = len(train_loader) * num_epochs
                #scheduler = torch.optim.lr_scheduler.LinearLR(
                #                                              optimizer,
                #                                              start_factor = 0.1,
                #                                              #total_iters = warmup_steps,
                #                                              total_iters = total_steps, # Fix: corrected from warmup_steps to total_steps
                #                                              )

                scheduler = get_linear_schedule_with_warmup(
                                                            optimizer,
                                                            num_warmup_steps = warmup_steps,
                                                            num_training_steps = total_steps
                                                            )

                # fine-tune model on training folds (x4)

                model.train()
                criterion = CrossEntropyLoss(weight = torch.tensor(
                                                                   class_weights,
                                                                   dtype = torch.float
                                                                   ).cuda() if use_cuda else torch.tensor(
                                                                                                          class_weights,
                                                                                                          dtype = torch.float
                                                                                                          )
                                             )

                for epoch in range(num_epochs):
                    for i, batch in enumerate(train_loader):
                        input_ids, attention_mask, labels = batch
                        labels = labels.long()

                        if use_cuda:
                            input_ids, attention_mask, labels = input_ids.cuda(), attention_mask.cuda(), labels.cuda()

                        outputs = model(input_ids, attention_mask = attention_mask)
                        logits = outputs.logits
                        loss = criterion(logits, labels)

                        # accumulate gradients, normalize loss

                        loss = loss / gradient_accumulation_steps
                        loss.backward()

                        # update model weights post-accumulation steps

                        if (i + 1) % gradient_accumulation_steps == 0:
                            optimizer.step()
                            optimizer.zero_grad()

                        # apply learning rate scheduler

                        scheduler.step()

                # evaluate on validation fold (x1)

                model.eval()
                all_predictions, all_true_labels = [], []
                with torch.no_grad():
                    for batch in valid_loader:
                        input_ids, attention_mask, labels = batch

                        if use_cuda:
                            input_ids, attention_mask, labels = input_ids.cuda(), attention_mask.cuda(), labels.cuda()

                        outputs = model(input_ids, attention_mask = attention_mask)
                        logits = outputs.logits
                        predictions = torch.argmax(logits, dim = 1).tolist()
                        all_predictions.extend(predictions)
                        all_true_labels.extend(labels.tolist())

                # performance metrics per validation fold

                f1_macro = f1_score(
                                    all_true_labels,
                                    all_predictions,
                                    average = 'macro',
                                    )

                mcc = matthews_corrcoef(
                                        all_true_labels,
                                        all_predictions,
                                        )

                auprc = average_precision_score(
                                                all_true_labels,
                                                all_predictions,
                                                average = 'macro',
                                                )

                fold_f1.append(f1_macro)
                fold_mcc.append(mcc)
                fold_auprc.append(auprc)

            # mean results over folds, track best model

            mean_f1 = np.mean(fold_f1)
            if mean_f1 > best_f1_scores[target]['score']:
                best_f1_scores[target]['score'] = mean_f1
                best_f1_scores[target]['model'] = model_name
                best_f1_scores[target]['model_instance'] = model

                save_model_name = f'{target}_{model_name}_best_{cycle}_model.pt'
                torch.save(model.state_dict(), save_path + save_model_name)

            # store results for each fold

            for i in range(5):
                results.append({
                                'target': target,
                                'model': model_name,
                                'fold': i + 1,
                                'f1_macro': fold_f1[i],
                                'mcc': fold_mcc[i],
                                'auprc': fold_auprc[i]
                                })

        # test on held-out d_test_{target} df

        print(f"\nTest on held-out d_test_{target} using the best {best_f1_scores[target]['model']} model")
        print("--------------------------------------------------------------------------------------")
        test_model = best_f1_scores[target]['model_instance']
        test_model.eval()

        #tokenizer = tokenizer_class.from_pretrained(pretrained_model_name)
        tokenizer = models[best_f1_scores[target]['model']][1].from_pretrained(pretrained_model_name)  # Fix: ensure correct tokenizer for testing
        encoded_test = tokenizer(
                                 X_test.tolist(),
                                 padding = True,
                                 truncation = True,
                                 return_tensors = 'pt',
                                 )

        test_dataset = TensorDataset(
                                     encoded_test['input_ids'],
                                     encoded_test['attention_mask'],
                                     torch.tensor(y_test),
                                     )

        test_loader = DataLoader(
                                 test_dataset,
                                 batch_size = batch_size,
                                 shuffle = False,
                                 )

        all_test_predictions, all_test_true_labels = [], []

        with torch.no_grad():
            for batch in test_loader:
                input_ids, attention_mask, labels = batch

                if use_cuda:
                    input_ids, attention_mask, labels = input_ids.cuda(), attention_mask.cuda(), labels.cuda()

                outputs = test_model(input_ids, attention_mask = attention_mask)
                logits = outputs.logits
                test_predictions = torch.argmax(logits, dim = 1).tolist()
                all_test_predictions.extend(test_predictions)
                all_test_true_labels.extend(labels.tolist())

        # preformance metrics for held-out d_test_{target} df

        test_f1_macro = f1_score(
                                 all_test_true_labels,
                                 all_test_predictions,
                                 average='macro',
                                 )

        test_mcc = matthews_corrcoef(
                                     all_test_true_labels,
                                     all_test_predictions,
                                     )

        test_auprc = average_precision_score(
                                             all_test_true_labels,
                                             all_test_predictions,
                                             average = 'macro',
                                             )

        # display

        print(f"Test F1 (macro) for {target}: {test_f1_macro}")
        print(f"Test MCC for {target}: {test_mcc}")
        print(f"Test AUPRC for {target}: {test_auprc}")

        # store

        results.append({
                        'target': target,
                        'model': best_f1_scores[target]['model'],
                        'fold': 'Test',
                        'f1_macro': test_f1_macro,
                        'mcc': test_mcc,
                        'auprc': test_auprc
                        })

    # summarize + return d_{cycle}_performance df

    print("\n--------------------------------------------------------------------------------------")
    print(f"Summary")
    print("--------------------------------------------------------------------------------------")

    for target, info in best_f1_scores.items():
        print(f"Best F1 (macro) for {target}: {info['score']} achieved by {info['model']}")

    d_performance = pd.DataFrame(results)
    print(f"\nd_{cycle}_performance:")
    print(d_performance.head(5))
    d_performance.to_excel(f'{save_path}d_{cycle}_performance.xlsx')

**_performance_barplot_**

In [ ]:
%%writefile -a train.py

import matplotlib.pyplot as plt
import seaborn as sns

def performance_barplot(df, plot_name):
    """
    Creates a barplot based on train_eval_save_bl_models d_{cycle}_performance output.

    Parameters:
    -----------
    df : pd.DataFrame
        The input dataframe that should contain columns 'target', 'f1_macro', and 'model'.

    plot_name : str
        The name of the dataframe (used for naming the saved file).

    Returns:
    --------
    Matplotlib Axes object containing the barplot.
    """

    model_colors = [
                    'paleturquoise',   # BERT
                    'palegreen', # RoBERTa
                    'lightpink', # DistilBERT
                    ]

    fig, ax = plt.subplots(
                           figsize=(
                                    14,   # width
                                    5.5,   # height
                                    )
                           )


    ax = sns.barplot(
                     data = df,
                     x = 'target',
                     y = 'f1_macro',
                     hue = 'model',
                     saturation = 0.75,
                     dodge = 'auto',
                     palette = model_colors,
                     linewidth = 0.5,
                     alpha = 0.8,
                     #edgecolor = 'gray',
                     errcolor = 'darkgray',
                     errwidth = 0.60,
                     capsize = 0.04,
                     )

    # legend

    ax.legend(
              loc = 'upper left',
              bbox_to_anchor = (
                                0.35,
                                1.1,
                                ),
              ncol = 3,
              #fancybox = True,
              #shadow = True
              title = None,
              frameon = False,
              )

    plt.ylim(
             0,
             1,
             )

    sns.set_style(
                  style = 'whitegrid',
                  rc = None,
                  )

    sns.despine(
                left = True,
                )


    # label axes

    ax.set_ylabel(
                  '$F_1$ (macro)',
                  fontsize = 14,
                  labelpad = 10,
                  )

    ax.set_xlabel(
                  'Target',
                  fontsize = 14,
                  labelpad = 10,
                  )

    plt.setp(ax.get_legend().get_texts(), fontsize = 12)
    plt.yticks(fontsize = 12)

    ax.set_xticklabels(
                       labels = [
                                 'asp',
                                 'dep',
                                 'val',
                                 'prg',
                                 'tgd',
                                 'age',
                                 'race',
                                 'dbty',
                                ],
                        #rotation = 45,
                        horizontalalignment = 'right',
                        fontsize = '12',
                        )

    # label bars

    for i in ax.containers:
        ax.bar_label(
                     i,
                     fmt = '%.2f',
                     label_type = 'edge',
                     #color = 'red',
                     rotation = 45,
                     fontsize = 8,
                     padding = 5,
                     )
    # save

    file_name = f'{plot_name}_bar.png'
    plt.savefig(file_name)

    # display

    plt.show()

    return ax

**_performance_scatterplot_**

In [ ]:
%%writefile -a train.py

import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib.lines import Line2D

def performance_scatterplot(df, plot_name):
    """
    Creates a categorical scatterplot with custom aesthetics, markers, error bars, and a legend.

    Parameters:
    df (pd.DataFrame): Input dataframe containing columns 'target', 'f1_macro', 'model', and 'fold'.

    plot_name (str): The name used for saving the output plot file (without extension).

    Returns:
    --------
    Matplotlib Axes object containing the barplot.
    """

    # aesthetics

    model_colors = [
                    '#87bc45',   # BERT
                    '#27aeef',   # RoBERTa
                    '#b33dc6',   # DistilBERT
                   ]

      ### SJS 10/1: last three colors in "Retro Metro (Default)" https://www.heavy.ai/blog/12-color-palettes-for-telling-better-stories-with-your-data

    sns.set_style(
                  style = 'whitegrid',
                  rc = None,
                  )

    # map target: numeric position

    target_mapping = {
                      'asp': 0,
                      'dep': 2,
                      'val': 4,
                      'prg': 6,
                      'tgd': 8,
                      'age': 10,
                      'race': 12,
                      'dbty': 14
                     }

    df['target_numeric'] = df['target'].map(target_mapping)
    df['target_numeric'] = pd.to_numeric(df['target_numeric'])

    # inject noise for jitter

    df['target_jitter'] = df['target_numeric'] + np.random.uniform(
                                                                   -0.35,
                                                                   0.35,
                                                                   size = len(df),
                                                                   )

    # plot

    plt.figure(figsize = (12, 5.5))

    # distinguish markers: fold v. held-out test set

    for fold_value, marker in [('Test', 'o'), ('non-Test', '.')]:
        if fold_value == 'Test':
            data_subset = df[df['fold'] == 'Test']
        else:
            data_subset = df[df['fold'] != 'Test']

        sns.scatterplot(
                        data = data_subset,
                        x = 'target_jitter',
                        y = 'f1_macro',
                        hue = 'model',
                        palette = model_colors,
                        s = 40,
                        alpha = 0.6,
                        marker = marker,
                       )

    # mean and SD of f1_macro for each target x model

    mean_std_df = df.groupby(['target', 'model']).agg(
                                                      mean_f1_macro=('f1_macro', 'mean'),
                                                      std_f1_macro=('f1_macro', 'std')
                                                      ).reset_index()

    # add target_numeric values to mean_std_df for plotting means and error bars

    mean_std_df['target_numeric'] = mean_std_df['target'].map(target_mapping).astype(float)

    # x-axis offsets

    model_offsets = {
                     'bert-base-uncased': -0.3,
                     'roberta-base': 0.0,
                     'distilbert-base-uncased': 0.3,
                     }

    mean_std_df['target_offset'] = mean_std_df['target_numeric'] + mean_std_df['model'].map(model_offsets)

    # means (SDs)

    for model in mean_std_df['model'].unique():
        model_data = mean_std_df[mean_std_df['model'] == model]

    # inspect for NaNs

        if not model_data[['target_offset', 'mean_f1_macro', 'std_f1_macro']].isnull().any().any():
            plt.errorbar(
                         model_data['target_offset'],
                         model_data['mean_f1_macro'],
                         yerr = model_data['std_f1_macro'],
                         fmt = 'D',
                         markersize = 6,
                         capsize = 0,
                         label = f'{model} M (SD)',
                         color = model_colors[mean_std_df['model'].unique().tolist().index(model)]
                        )

    # x-tick: map to targets

    plt.xticks(
               [0, 2, 4, 6, 8, 10, 12, 14],
               ['asp', 'dep', 'val', 'prg', 'tgd', 'age', 'race', 'dbty']
              )

    # label axes

    plt.ylim(0, 1)
    ax = plt.gca()
    ax.set_ylabel(
                  '$F_1$ (macro)',
                  fontsize = 12,
                  labelpad = 10,
                  )

    ax.set_xlabel(
                  'Target',
                  fontsize = 12,
                  labelpad = 10,
                  )

    sns.despine(left = True)
    ax.grid(axis = 'x')

    # set line at 0.9 threshold

    ax.axhline(
               y = 0.9,
               color = 'r',
               linewidth = 0.6,
               linestyle = '--',
               )

    # custom legend

    legend_elements = [
                       Line2D([0], [0], marker = 'o', color = 'w', label = 'bert', markersize = 8, markerfacecolor = '#87bc45', lw = 0),
                       Line2D([0], [0], marker = 'o', color = 'w', label = 'roberta', markersize = 8, markerfacecolor = '#27aeef', lw = 0),
                       Line2D([0], [0], marker = 'o', color = 'w', label = 'distilbert', markersize = 8, markerfacecolor = '#b33dc6', lw = 0),
                       Line2D([0], [0], marker = 'D', color = 'black', label = 'M (SD)', markersize = 7, lw = 1, linestyle = '-', markeredgewidth = 1)
                      ]

    ax.legend(
              handles = legend_elements,
              loc = 'upper center',
              bbox_to_anchor = (0.5, 1.15),
              ncol = 4,
              fontsize = 9,
              frameon = False,
              )

    # save

    file_name = f'{plot_name}_scatter.png'
    plt.savefig(file_name)

    # display
    plt.show()


**_iterative_stratified_train_test_split_with_rationales_**

In [ ]:
%%writefile -a train.py

import pandas as pd
from sklearn.model_selection import train_test_split

def iterative_stratified_train_test_split_with_rationales(df, targets, test_size, random_state):
    """
    Splits df into target-stratified train and test sets for each target in targets list:
    d_train_{target}, d_test_{target}, respectively. Partitions 'rationales' (aug = 1) to train
    set. Returns a dict with target names as keys
    """

    # initialize dict

    target_datasets = {}

    for target in targets:

        # create 'targets' col for stratification

        df_target = df.copy()
        df_target['targets'] = df[target]

        # split augmented vs. non-augmented rows

        aug_rows = df_target[df_target['aug'] == 1]
        non_aug_rows = df_target[df_target['aug'] != 1]

        if non_aug_rows.empty:
            print(f"No non-augmented rows for target {target}. Skipping...")
            continue

        # stratified train-test split on non-augmented rows only

        train_non_aug, test_non_aug = train_test_split(
                                                       non_aug_rows,
                                                       test_size = test_size,
                                                       stratify = non_aug_rows['targets'],
                                                       random_state = random_state,
                                                      )

        # concat augmented rows back into train set

        d_train = pd.concat([train_non_aug, aug_rows])

        # shuffle + reset index: train set

        d_train = d_train.sample(
                                 frac = 1,
                                 random_state = random_state,
                                 ).reset_index(drop = True)

        # retain 'text', 'aug', target cols

        d_train = d_train[['text', 'aug', target]]
        d_test = test_non_aug[['text', 'aug', target]]

        # reset index: test set

        d_test = d_test.reset_index(drop = True)

        # add train and test sets as tuples to target_datasets dict

        target_datasets[target] = (d_train, d_test)

        # inspect

        print(f"\nVerify: d_train_{target} 'aug' count")
        print(d_train['aug'].value_counts(normalize = False))
        print(f"\nVerify: d_test_{target} 'aug' count")
        print(d_test['aug'].value_counts(normalize = False))

        print(f"\n--------------------------------------------------------------------------------------")
        print(f"d_train_{target}: Augmented training data for target '{target}'")
        print(f"--------------------------------------------------------------------------------------")
        print(d_train.shape)
        print(d_train[target].value_counts(normalize = True))
        print(d_train.head(6))

        print(f"\n--------------------------------------------------------------------------------------")
        print(f"d_test_{target}: De-augmented testing data for target '{target}'")
        print(f"--------------------------------------------------------------------------------------")
        print(d_test.shape)
        print(d_test[target].value_counts(normalize = True))
        print(d_test.head(6))

    return target_datasets


**_tune_and_optimize_model_hyperparams_**

In [ ]:
%%writefile -a train.py

import torch
from torch.utils.data import DataLoader, TensorDataset
from torch.nn import CrossEntropyLoss
from transformers import get_linear_schedule_with_warmup
from sklearn.metrics import f1_score
from sklearn.model_selection import ParameterGrid
from tqdm import tqdm
import pandas as pd

def tune_and_optimize_model_hyperparams(tokenizer, model_class, pretrained_model_name, d_train, d_test, target, class_weights, save_path, hyperparameter_grid):
    """
    Tune and optimize model hyperparameters for a specific model-target combination.

    Parameters:
    -----------

    tokenizer:
        Pre-trained tokenizer.

    model_class:
        Pre-trained model class.

    pretrained_model_name:
        Name of the pre-trained model.

    d_train : pd.DataFrame
        Training dataset.

    d_test : pd.DataFrame
        Test dataset.

    target : str
        The target variable for classification.

    class_weights : torch.tensor
        Weights for each class.

    save_path : str
        Path to save the best model.

    hyperparameter_grid : dict
        Dictionary where keys are hyperparameter names and values are lists of possible values.

    Returns:
    --------
    d_test : pd.DataFrame
        Test dataset with predictions and probabilities.

    d_tuned_performance : pd.DataFrame
        DataFrame with the performance metrics for each hyperparameter configuration.
    """

    # check CUDA

    use_cuda = torch.cuda.is_available()
    print("CUDA: ", use_cuda)

    # set seed

    set_seed(56)

    print("======================================================================================")
    print(f"Optimizing: {pretrained_model_name}\nTarget: {target}")
    print("======================================================================================")

    # tokenize train and test sets

    encoded_train = tokenizer(
                              d_train['text'].tolist(),
                              padding = True,
                              truncation = True,
                              return_tensors = 'pt',
                              )

    encoded_test = tokenizer(
                             d_test['text'].tolist(),
                             padding = True,
                             truncation = True,
                             return_tensors = 'pt',
                             )


    # accept dynamic target variables

    train_labels = torch.tensor(d_train[target].values)
    test_labels = torch.tensor(d_test[target].values)

    # prepare datasets

    train_dataset = TensorDataset(
                                  encoded_train['input_ids'],
                                  encoded_train['attention_mask'],
                                  train_labels,
                                  )

    test_dataset = TensorDataset(
                                 encoded_test['input_ids'],
                                 encoded_test['attention_mask'],
                                 test_labels,
                                 )

    #train_loader = DataLoader(
    #                          train_dataset,
    #                          batch_size = 8,  ### to be updated within grid search
    #                          shuffle = True,
    #                          )

    #test_loader = DataLoader(
    #                         test_dataset,
    #                         batch_size = 8,  ### to be updated within grid search
    #                         shuffle = False,
    #                         )

    # initialize class weights

    if use_cuda:
        class_weights = class_weights.cuda()

    # initialize tracking variables

    best_f1_macro = 0
    best_params = None
    best_model_state = None
    best_predictions = []
    best_probabilities = []

    f1_scores = []
    performance_data = []

    # hyperparam grid search: ParameterGrid

    for hyperparams in ParameterGrid(hyperparameter_grid):
        print(f"\nOptimizing with hyperparameters: {hyperparams}")

        train_loader = DataLoader(
                                  train_dataset,
                                  batch_size = hyperparams['batch_size'],
                                  shuffle = True
                                  )
        test_loader = DataLoader(
                                 test_dataset,
                                 batch_size = hyperparams['batch_size'],
                                 shuffle = False
                                 )

        print(f"\nTotal training rows: {len(train_dataset)}")
        print(f"Total evaluation rows: {len(test_dataset)}")
        print(f"Training batch size: {hyperparams['batch_size']}")
        print(f"Evaluation batch size: {hyperparams['batch_size']}")
        print(f"Total training batches: {len(train_loader)}")
        print(f"Total evaluation batches: {len(test_loader)}")
        print("\n")

        # initialize model

        model = model_class.from_pretrained(pretrained_model_name)
        if use_cuda:
            model.cuda()

        # initialize optimizer and lr scheduler

        optimizer = torch.optim.AdamW(
                                      model.parameters(),
                                      lr = hyperparams['learning_rate'],
                                      weight_decay = hyperparams['weight_decay']
                                      )

        # calculate total steps

        total_steps = len(train_loader) * hyperparams['num_epochs']

        # add scheduler with warmup steps

        scheduler = get_linear_schedule_with_warmup(
                                                    optimizer,
                                                    num_warmup_steps = hyperparams['warmup_steps'],
                                                    num_training_steps=total_steps
                                                    )

        criterion = CrossEntropyLoss(weight = class_weights)

        # training loop

        for epoch in range(hyperparams['num_epochs']):
            model.train()
            optimizer.zero_grad()
            for i, batch in enumerate(tqdm(train_loader, desc = f"Training Epoch {epoch + 1}/{hyperparams['num_epochs']}", leave=True)):
                input_ids, attention_mask, labels = batch
                if use_cuda:
                    input_ids, attention_mask, labels = input_ids.cuda(), attention_mask.cuda(), labels.cuda()
                outputs = model(input_ids, attention_mask=attention_mask)
                loss = criterion(outputs.logits, labels)
                loss = loss / hyperparams['gradient_accumulation_steps']
                loss.backward()
                if (i + 1) % hyperparams['gradient_accumulation_steps'] == 0:
                    optimizer.step()
                    scheduler.step()  ### update learning rate
                    optimizer.zero_grad()

        # eval loop

        model.eval()
        all_predictions = []
        all_true_labels = []
        all_probabilities = []
        with torch.no_grad():
            for batch in test_loader:
                input_ids, attention_mask, labels = batch
                if use_cuda:
                    input_ids, attention_mask, labels = input_ids.cuda(), attention_mask.cuda(), labels.cuda()
                outputs = model(input_ids, attention_mask=attention_mask)
                probabilities = torch.softmax(outputs.logits, dim = 1)
                predictions = torch.argmax(probabilities, dim = 1).cpu().tolist()
                all_predictions.extend(predictions)
                all_true_labels.extend(labels.cpu().tolist())
                all_probabilities.extend(probabilities.cpu().tolist())

        # calculate F1 (macro)

        current_f1_macro = f1_score(all_true_labels, all_predictions, average='macro')
        f1_scores.append(current_f1_macro)
        print(f"\nCurrent F1 macro with params {hyperparams}: {current_f1_macro}")

        # append F1 and current performance data

        performance_data.append({
                                 'pretrained_model_name': pretrained_model_name,
                                 'target': target,
                                 'f1_score': current_f1_macro,
                                 'batch_size': hyperparams['batch_size'],
                                 'weight_decay': hyperparams['weight_decay'],
                                 'learning_rate': hyperparams['learning_rate'],
                                 'warmup_steps': hyperparams['warmup_steps'],
                                 'num_epochs': hyperparams['num_epochs'],
                                 'gradient_accumulation_steps': hyperparams['gradient_accumulation_steps'],
        })

        if current_f1_macro > best_f1_macro:
            best_f1_macro = current_f1_macro
            best_params = hyperparams
            best_model_state = model.state_dict()
            best_predictions = all_predictions
            best_probabilities = all_probabilities

    #if len(best_predictions) == len(d_test):
    #    d_test['predicted_labels'] = best_predictions
    #    d_test['predicted_probabilities'] = best_probabilities
    #else:
    #    print("Error: Length of predictions does not match length of test set")

    d_test['predicted_labels'] = best_predictions
    d_test['predicted_probabilities'] = best_probabilities

    # save d_test_{target} with pred and prob

    print("--------------------------------------------------------------------------------------")
    print(f"Summary: {target}")
    print("--------------------------------------------------------------------------------------")

    print(d_test.head(6))
    d_test.to_excel(f'{save_path}/d_test_tuned_preds_{target}.xlsx')

    if best_model_state:
        model_path = f"{save_path}/{target}_{pretrained_model_name}_best_tuned_model.bin"
        torch.save(best_model_state, model_path)
        print("\nBest model saved with F1 macro:", best_f1_macro)
        print("Best hyperparameters:", best_params)

    # display F1 scores

    f1_mean = sum(f1_scores) / len(f1_scores)
    f1_std = (sum((x - f1_mean) ** 2 for x in f1_scores) / len(f1_scores)) ** 0.5
    print(f"Mean F1 macro: {f1_mean}")
    print(f"Standard deviation of F1 macro: {f1_std}")

    # df: target-wise

    d_tuned_performance = pd.DataFrame(performance_data)
    print(d_tuned_performance.head(10))

    # save: target-wise df

    d_tuned_performance.to_excel(f'{save_path}/d_tuned_performance_{target}.xlsx')

    return d_test, d_tuned_performance


#### redact.py

**_ner_redact_post_texts_**

In [ ]:
%%writefile redact.py

import spacy
nlp = spacy.load('en_core_web_lg')

def ner_redact_post_texts(p_text):
    """
    Redacts all named entities recognized by spaCy EntityRecognizer, replaces with <|PII|> pseudo-word token.
    """
    ne = list(
              [
               'PERSON',   ### people, including fictional
               'NORP',     ### nationalities or religious or political groups
               'FAC',      ### buildings, airports, highways, bridges, etc.
               'ORG',      ### companies, agencies, institutions, etc.
               #'GPE',     ### countries, cities, states
               'LOC',      ### non-GPE locations, mountain ranges, bodies of water
               'PRODUCT',  ### objects, vehicles, foods, etc. (not services)
               'EVENT',    ### named hurricanes, battles, wars, sports events, etc.
               ]
                )

    doc = nlp(p_text)
    ne_to_remove = []
    final_string = str(p_text)
    for sent in doc.ents:
        if sent.label_ in ne:
            ne_to_remove.append(str(sent.text))
    for n in range(len(ne_to_remove)):
        final_string = final_string.replace(
                                            ne_to_remove[n],
                                            '<|PII|>',
                                            )
    return final_string

#### predict.py

**_load_model_**

In [ ]:
%%writefile a predict.py

import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import (
                          DistilBertTokenizer,
                          DistilBertForSequenceClassification,
                          BertTokenizer,
                          BertForSequenceClassification,
                          RobertaTokenizer,
                          RobertaForSequenceClassification,
                          )
from tqdm import tqdm
import pandas as pd

def load_model(model_path, model_class, pretrained_model_name):
    """
    Loads a pre-trained fine-tined LM from a specified path.
    """
    model = model_class.from_pretrained(pretrained_model_name)
    model.load_state_dict(torch.load(model_path))
    model.eval()
    return model

**_preprocess_data_**

In [ ]:
%%writefile -a predict.py

import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import (
                          DistilBertTokenizer,
                          DistilBertForSequenceClassification,
                          BertTokenizer,
                          BertForSequenceClassification,
                          RobertaTokenizer,
                          RobertaForSequenceClassification,
                          )
from tqdm import tqdm
import pandas as pd

def preprocess_data(tokenizer, texts):
    """
    Tokenizes a list of texts using the specified LM-specific tokenizer.
    """
    encoded_texts = tokenizer(
        texts,
        padding=True,
        truncation=True,
        return_tensors='pt'
    )
    return encoded_texts

**_predict_**

In [ ]:
%%writefile -a predict.py

import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import (
                          DistilBertTokenizer,
                          DistilBertForSequenceClassification,
                          BertTokenizer,
                          BertForSequenceClassification,
                          RobertaTokenizer,
                          RobertaForSequenceClassification,
                          )
from tqdm import tqdm
import pandas as pd

def predict(model, tokenizer, texts, batch_size = 8, use_cuda = True):
    """
    Predicts labels and probabilities for a list of texts using the specified model and tokenizer.
    """
    print(f"Total number of texts to predict: {len(texts)}")
    encoded_texts = preprocess_data(tokenizer, texts)
    dataset = TensorDataset(encoded_texts['input_ids'], encoded_texts['attention_mask'])
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    print(f"Batch size: {batch_size}")
    print(f"Total number of batches: {len(data_loader)}")

    if use_cuda:
        model.cuda()

    all_predictions = []
    all_probabilities = []

    with torch.no_grad():
        progress_bar = tqdm(total=len(data_loader), desc="Predicting", leave=False)
        for batch in data_loader:
            input_ids, attention_mask = batch
            if use_cuda:
                input_ids, attention_mask = input_ids.cuda(), attention_mask.cuda()

            outputs = model(input_ids, attention_mask=attention_mask)
            probabilities = torch.softmax(outputs.logits, dim=1)
            predictions = torch.argmax(probabilities, dim=1).cpu().tolist()
            all_predictions.extend(predictions)
            all_probabilities.extend(probabilities.cpu().tolist())
            progress_bar.update(1)
        progress_bar.close()

    return all_predictions, all_probabilities

#### Import

In [ ]:
from preprocess import (
                        augment_training_data_with_rationales,
                        dummy_code_augmented_rows,
                        read_and_append_jsonl_archives,
                        )

#from redact import (
#                    ner_redact_post_texts,
#                    )

#from predict import (
#                     load_model,
#                     preprocess_data,
#                     predict,
#                     )

from train import (
                   set_seed,
                   train_eval_save_bl_models,
                   performance_barplot,
                   performance_scatterplot,
                   iterative_stratified_train_test_split_with_rationales,
                   tune_and_optimize_model_hyperparams,
                   )

### 3. Preprocess
Takes $\mathcal{d}$<sub>annotated</sub>, builds independent $\mathcal{d}$<sub>calibrate</sub> and rationale-augmented $\mathcal{d}$<sub>augmented</sub> to train. Builds $\mathcal{V}$ corpus, $\mathcal{d}$<sub>adapt</sub> for domain adaptation. Merges, NER-anonymizes Aim II analytic sample $\mathcal{D}$<sub>inference</sub>.
***

#### Merge Wave 1 (purposive) and Wave 2 (random)

In [ ]:
%cd ../inputs/data

In [ ]:
d_1 = pd.read_excel('d_cycle999_prp_ss_single.xlsx')
d_2 = pd.read_excel('d_cycle999_rnd_ss_single.xlsx')

d = pd.concat([
               d_1,
               d_1,
               ])

d = d.drop(
           'Unnamed: 0',
           axis = 1,
           )

d = d[d.text != ' ']

d.replace(
          ' ',
          0,
          inplace = True,
          )

ints = [
        'asp',
        'dep',
        'val',
        'prg',
        'tgd',
        'age',
        'race',
        'dbty',
        ]

for i in ints:
    d[i] = pd.to_numeric(d[i], errors = 'coerce')
    d[i] = d[i].fillna(0).astype('int64')

d_annotated = shuffle(
                      d,
                      random_state = 56,
                      )

d_annotated.reset_index(
                        drop = True,
                        inplace = True,
                        )

sbrt_cnt = d_annotated['sbrt'].value_counts()
sbrt_pct = d_annotated['sbrt'].value_counts(normalize = True) * 100

sbrts = pd.DataFrame({
                      'Count': sbrt_cnt,
                      'Percentage': sbrt_pct,
                      })

print(sbrts)

d_annotated[[
             'asp',
             'dep',
             'val',
             'prg',
             'tgd',
             'age',
             'race',
             'dbty',
              ]].apply(pd.Series.value_counts)

#d.dtypes
d_annotated.info()
d_annotated.head(3)

$\mathcal{d}$<sub>calibrate</sub> ($n$<sub>posts</sub> = 400)

In [ ]:
d_calibrate = d_annotated.iloc[:400]

d_calibrate.shape
d_calibrate.head(3)

d_calibrate.to_excel('d_calibrate.xlsx')

$\mathcal{d}$<sub>augmented</sub> (unique $n$<sub>posts</sub> = 2,000)

In [ ]:
d = d_annotated.iloc[400:]

d.reset_index(
              drop = True,
              inplace = True,
              )

# 'strn' = any pos_instance of strain

d['strn'] = (d['asp'] == 1) | (d['dep'] == 1) | (d['val'] == 1)
d['strn'] = d['strn'].astype(int)

# append rationales

rationales = [
              'asp_rtnl',
              'dep_rtnl',
              'val_rtnl',
               ]

for r in rationales:
    d[r] = d[r].astype(str)
    d[r] = d[r].str.replace(
                            r'0',
                            '.',
                            regex = True,
                            )

d['rtnl'] = d['asp_rtnl'] + ' ' + d['dep_rtnl'] + ' ' + d['val_rtnl']

d['rtnl'] = d['rtnl'].str.replace(
                                  r'. . .',
                                  '.',
                                  regex = False,
                                  )

print("pre-augmentation")
print("--------------------------------------------------------------------------------------")
d.shape
d.head(3)

In [ ]:
d_augmented = augment_training_data_with_rationales(d)

d_augmented.reset_index(
                        drop = True,
                        inplace = True,
                        )

d_augmented = dummy_code_augmented_rows(d_augmented)

print("post-augmentation")
print("--------------------------------------------------------------------------------------")

d_augmented[[
             'asp',
             'dep',
             'val',
             'prg',
             'tgd',
             'age',
             'race',
             'dbty',
             'aug'
              ]].apply(pd.Series.value_counts)

d_augmented.shape
d_augmented.head(30)

d_augmented.to_excel('d_augmented.xlsx')

In [ ]:
d_augmented = pd.read_excel(
                            'd_augmented.xlsx',
                            index_col = [0],
                            )

d_augmented[[
             'asp',
             'dep',
             'val',
             'prg',
             'tgd',
             'age',
             'race',
             'dbty',
             'aug'
              ]].apply(pd.Series.value_counts)

#### Append $\mathcal{V}$ corpus, derive $\mathcal{D}$<sub>inference</sub>

In [ ]:
archives_path = '/content/drive/MyDrive/Colab/bar_policy_suicidality/inputs/archives/'
#archives_path = '/content/drive/MyDrive/Colab/bar_policy_suicidality/inputs/archives/TEST'
d_p, d_c = read_and_append_jsonl_archives(archives_path)

    ### SJS 9/11: impossible RAM overhead using full corpus; solution TKTK

d_p.shape
d_p.head(3)

d_c.shape
d_c.head(3)

In [ ]:
############################################################
############################################################
############################################################ PRELIM (START)

%cd inputs/data

In [ ]:
%reset_selective -f d_

In [ ]:
# d_adapt 9/12 - r/SW, r/trans, r/TwoX 2022 posts only

    ### SJS 9/12: importing one by one to mitigate RAM overhead - long-term solution TKTK

archives_path = '/content/drive/MyDrive/Colab/bar_policy_suicidality/inputs/archives/'
list_p = []

p_path = f'{archives_path}r_TwoXChromosomes_posts.jsonl'

d_tx_p = pd.read_json(
                      p_path,
                      lines = True,
                     )

# inspect

d_tx_p.info()
d_tx_p.head(3)

# save

d_tx_p.to_csv(
              'd_tx_p.csv',
              encoding = 'utf-8',
              index = False,
              header = True,
              )

In [ ]:
d_tx_p = pd.read_csv('d_tx_p.csv')

d_tx_p = d_tx_p.drop_duplicates(subset = 'id')

# Check for non-numeric values in 'created_utc' column
#non_numeric = d_sw_p[pd.to_numeric(d_sw_p['created_utc'], errors='coerce').isnull()]

# Investigate or handle non-numeric values before converting to datetime
#if len(non_numeric) > 0:
#  print(f"Found {len(non_numeric)} non-numeric values in 'created_utc':")
#  print(non_numeric['created_utc'])
  # Consider removing or correcting these values before proceeding

#d_sw_p = d_sw_p.drop(45358)

d_tx_p['date'] = pd.to_datetime(
                             d_tx_p.created_utc,
                             unit = 's',
                             )

d_tx_p.set_index(
              'date',
              drop = False,
              inplace = True,
              )

d_tx_p = d_tx_p.loc[(d_tx_p['date'] >= '2022-01-01') & (d_tx_p['date'] < '2022-12-31')] ### 2022 for d_adapt

d_tx_p = d_tx_p[~d_tx_p['selftext'].isin(['[deleted]', '[removed]'])]

# housekeeping

d_tx_p = d_tx_p[[
           'author',
           'created_utc',
           'date',
           'id',
           'num_comments',
           'selftext',
           'subreddit',
           'title',
           ]].copy()

d_tx_p.rename(
           columns = {
                      'author': 'p_au',
                      'created_utc': 'p_utc',
                      'date': 'p_date',
                      'num_comments': 'n_cmnt',
                      'selftext': 'text',
                      'subreddit': 'p_sbrt',
                      'title': 'p_titl',
                      }, inplace = True,
            )

d_tx_p.shape
d_tx_p.head(3)

d_tx_p.to_csv(
              'd_tx_p.csv',
              encoding = 'utf-8',
              index = False,
              header = True,
              )


In [ ]:
d_sw_p.shape
d_sw_p.head(3)

d_tr_p.shape
d_tr_p.head(3)

d_tx_p.shape
d_tx_p.head(3)

In [ ]:
d_adapt = pd.concat([
                     d_sw_p,
                     d_tr_p,
                     d_tx_p,
                     ])

d_adapt = shuffle(d_adapt)

d_adapt.reset_index(
                    drop = True,
                    inplace = True,
                    )

d_adapt.shape
d_adapt.head(3)

d_adapt.to_csv(
              'd_adapt.csv',
              encoding = 'utf-8',
              index = False,
              header = True,
              )

############################################################ PRELIM (END)
############################################################
############################################################

**Clean, condense: posts**

In [ ]:
# d_p = posts

d_p = d_p.drop_duplicates(subset = 'id')

d_p['date'] = pd.to_datetime(
                             d_p.created_utc,
                             unit = 's',
                             )

d_p.set_index(
              'date',
              drop = False,
              inplace = True,
              )

d_p = d_p.loc[(d_p['date'] >= '2020-12-02') & (d_p['date'] < '2024-06-24')] ### yyyy-mm-dd = Dec 2, 2020 - Jun 24, 2024

d_p = d_p[~d_p['selftext'].isin(['[deleted]', '[removed]'])]

# housekeeping

d_p = d_p[[
           'author',
           'created_utc',
           'date',
           'id',
           'num_comments',
           'selftext',
           'subreddit',
           'title',
           ]].copy()

d_p.rename(
           columns = {
                      'author': 'p_au',
                      'created_utc': 'p_utc',
                      'date': 'p_date',
                      'num_comments': 'n_cmnt',
                      'selftext': 'text',
                      'subreddit': 'p_sbrt',
                      'title': 'p_titl',
                      }, inplace = True,
            )

**NER anonymize: posts**

In [ ]:
d_p['text'] = d_p['text'].astype(str).apply(lambda i: ner_redact_post_texts(i))

**Clean, condense: comments**

In [ ]:
# d_c = comments

d_c = d_c.drop_duplicates(subset = 'id')

d_c['date'] = pd.to_datetime(
                             d_c.created_utc,
                             unit = 's',
                             )

d_c.set_index(
              'date',
              drop = False,
              inplace = True,
              )

d_c = d_c.loc[(d_c['date'] >= '2020-12-02') & (d_c['date'] < '2024-06-24')] ### yyyy-mm-dd = Dec 2, 2020 - Jun 24, 2024

d_c = d_c[~d_c['body'].isin(['[deleted]', '[removed]'])]

# housekeeping

d_c = d_c[[
           'author',
           'body',
           'date',
           'link_id',
           'subreddit',
           ]].copy()

d_c.rename(
           columns = {
                      'author': 'c_au',
                      'body': 'c_text',
                      'date': 'c_date',
                      'link_id': 'id',
                      'subreddit': 'c_sbrt',
                      }, inplace = True,
            )

# delete comment-level 'id' prefix for merge

d_c['id'] = d_c['id'].str.replace('t3_', ' ')

In [ ]:
print("V-corpus posts/d_adapt")
print("--------------------------------------------------------------------------------------")
d_p.shape
d_p.head(3)
d_p.tail(3)

print("V-corpus comments")
print("--------------------------------------------------------------------------------------")
d_c.shape
d_c.head(3)
d_c.tail(3)

In [ ]:
# for post-labeling merge

d_c.to_csv(
           'd_comments.csv',
           encoding = 'utf-8',
           index = False,
           header = True,
           )

$\mathcal{d}$<sub>adapt</sub>: domain adaptation set

In [ ]:
d_adapt = d_p.copy()

d_adapt.to_csv(
               'd_adapt_TEST.csv',
               encoding = 'utf-8',
               index = False,
               header = True,
               )

$\mathcal{D}$<sub>inference</sub>: prediction set

In [ ]:
d_inference = d_p.sample(
                         n = 1000000, ### TKTK - maybe
                         random_state = 56,
                         )

**GPE: encoding, extraction**

In [ ]:
nlp = spacy.load('en_core_web_lg')

# extract, count GPEs

def extract_gpe(text):
    doc = nlp(text)
    gpes = [ent.text for ent in doc.ents if ent.label_ == 'GPE']
    return gpes, len(gpes)

d_inference[[
             'gpe',
             'gpe_count',
             ]] = d_inference['text'].apply(lambda i: pd.Series(extract_gpe(i)))

total_gpe_count = d_inference['gpe_count'].sum()

print(f"Total number of GPEs recognized: {total_gpe_count}")

In [ ]:
    ### SJS 8/17: validation w/ GTP-4o TKTK: is each GPE in the US?

**GPE: concordance**

In [ ]:
# join single string for nltk entry

all_texts = ' '.join(d_inference['text'].tolist())

# tokenize for nltk

tokens = nltk.word_tokenize(all_texts)
nltk_text = Text(tokens)

# concordance: GPEs in context

for gpes in d_inference['gpe']:
    for gpe in gpes:  # gpes = list of GPEs
        print(f"\nConcordance for '{gpe}':")
        nltk_text.concordance(gpe)

**Explicit suicidality: encoding**

In [ ]:
regex = r'\bsuicid\S*'

d_inference['sui'] = d_inference['text'].str.contains(
                                                      regex,
                                                      regex = True,
                                                      ) | (d_inference['p_sbrt'] == 'SuicideWatch')


d_inference['sui'] = d_inference['sui'].astype(int)

d_inference.head(3)

In [ ]:
    ### SJS 8/17: validation w/ GTP-4o TKTK: is sui in reference to other people?

#%pwd
%cd ../inputs/data

In [ ]:
d_inference.to_csv(
                   'd_inference_TEST.csv',
                   encoding = 'utf-8',
                   index = False,
                   header = True,
                   )

### 4. Train-Test
Trains baseline BERT, RoBERTa, and DistilBERT, using rationale-augmented data $\mathcal{d}$<sub>augmented</sub>, iterating over a.) strains, b.) explicit targeting, c.) implicit vulnerabilities. Evaluates using de-augmented data. Outputs model x target _$F$_<sub>1</sub> (macro) performance scores.
***

In [ ]:
#%pwd
%cd ../inputs/data

In [ ]:
d_augmented = pd.read_excel(
                            'd_augmented.xlsx',
                            index_col = [0],
                            )

d_augmented.info()
d_augmented.head(3)

**Condense for model entry**

In [ ]:
targets = [
           'asp',
#           'dep',
#           'val',
#           'prg',
#           'tgd',
#           'age',
#           'race',
#           'dbty',
           ]

d_augmented = d_augmented[
                          ['text',
                           'aug'] +
                           targets
                           ].copy()

d_augmented[
            ['aug'] +
             targets
             ].apply(pd.Series.value_counts)

**Compute weights ($w$): inverse class ($c$) freq<br>
$w_c = N / (2 * n_c)$**

In [ ]:
class_weights = {}

for t in targets:

    value_counts = d_augmented[t].value_counts()

    w_pos = round(len(d_augmented) / (2 * value_counts.get(1, 0)), 4)
    w_neg = round(len(d_augmented) / (2 * value_counts.get(0, 0)), 4)

    class_weights[t] = {
                        'w_pos': w_pos if w_pos != float('inf') else 0,
                        'w_neg': w_neg if w_neg != float('inf') else 0,
                        }

class_weights

#### Train and evaluate benchmark models: $k$-fold cross validate

In [ ]:
%cd /content/drive/My Drive/Colab/bar_policy_suicidality/temp

# define target-specific aug-stratified df

target_datasets = iterative_stratified_train_test_split_with_rationales(
                                                                        d_augmented,
                                                                        targets,
                                                                        random_state = 56,
                                                                        test_size = 0.2,
                                                                        )

In [ ]:
print(target_datasets.keys())

In [ ]:
# define target-specific df

#target_datasets = {
#                   'asp': (d_train_asp, d_test_asp),
#                   'dep': (d_train_dep, d_test_dep),
#                   'val': (d_train_val, d_test_val),
#                   'prg': (d_train_prg, d_test_prg),
#                   'tgd': (d_train_tgd, d_test_tgd),
#                   'age': (d_train_age, d_test_age),
#                   'race': (d_train_race, d_test_race),
#                   'dbty': (d_train_dbty, d_test_dbty),

#}

# define targets + class weights

targets_and_class_weights = {
                             'asp': [
                                     0.7657, ### w_neg
                                     1.4410, ### w_pos
                                     ],
#                             'dep': [
#                                     0.5989,
#                                     3.0286,
#                                     ],
#                             'val': [
#                                     0.6876,
#                                     1.8327,
#                                     ],
#                             'prg': [
#                                     0.6024,
#                                     2.9414,
#                                     ],
#                            'tgd': [
#                                     0.5974,
#                                     3.0676,
#                                     ],
#                             'age': [
#                                     0.6303,
#                                     2.4188,
#                                     ],
#                             'race': [
#                                      0.506,
#                                      42.0441,
#                                      ],
#                             'dbty': [
#                                      0.5064,
#                                      39.7083,
#                                      ],
                              }

**BERT, RoBERTa, DistilBERT**

In [ ]:
# define models

models = {
          'bert': (
                   BertForSequenceClassification,
                   BertTokenizer,
                  'bert-base-uncased',
                   ),

          'roberta': (
                      RobertaForSequenceClassification,
                      RobertaTokenizer,
                      'roberta-base',
                      ),

#          'distilbert': (
#                         DistilBertForSequenceClassification,
#                         DistilBertTokenizer,
#                         'distilbert-base-uncased',
#                         ),
        }

save_path = '/content/drive/MyDrive/Colab/bar_policy_suicidality/outputs/models/'

# define hyperparameter grid

hyperparameter_grid = {
                       'batch_size': 8,
                       'gradient_accumulation_steps': 2,
                       'learning_rate': 2e-5,
                       'num_epochs': 2,
                       'warmup_steps': 0,
                       'weight_decay': 0.00,
                       }

# set cycle

cycle = 'benchmark'

In [ ]:
%cd ../../outputs/tables

# 'benchmark' cycle: train-test loop

train_eval_save_bl_models(
                          target_datasets = target_datasets,
                          targets_and_class_weights = targets_and_class_weights,
                          models = models,
                          save_path = save_path,
                          cycle = cycle,
                          hyperparameter_grid = hyperparameter_grid,
                          )

**Llama**

In [ ]:
##############################################################################
##############################################################################
############################################################################## PRELIM

#%pip install llama-stack

In [ ]:
!huggingface-cli login --token '<my_token>'

In [ ]:
############################################################################## PRELIM
##############################################################################
##############################################################################

**Benchmark: viz**

In [ ]:
#from matplotlib.font_manager import get_font_names
#print(get_font_names())

%cd /content/drive/MyDrive/Colab/bar_policy_suicidality/temp

In [ ]:
%%capture

d_v = pd.read_excel('d_benchmark_performance_SHAM.xlsx')
d_v.round({'f1_macro': 4, 'mcc': 4, 'auprc': 4})

In [ ]:
%cd ../figures

performance_barplot(
                    d_v,
                    'benchmark_performance',
                    )

#### Adapt

In [ ]:
%cd /content/drive/MyDrive/Colab/bar_policy_suicidality/inputs/data
#%cd inputs/data

In [ ]:
d_adapt = pd.read_csv('d_adapt.csv')

d_adapt.info()
d_adapt.head(3)

In [ ]:
d_adapt = d_adapt[[
                   'text',
                   'p_sbrt',
                   ]].copy()

d_adapt.shape
d_adapt.head(3)

In [ ]:
%cd ../../outputs/models
models_path = '/content/drive/MyDrive/Colab/bar_policy_suicidality/outputs/models/'

**Domain adaptation proxy task: subreddit clr**

In [ ]:
# prep dataset

class CustomDataset(Dataset):
    def __init__(self, df, tokenizer, max_length):
        self.df = df
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        text = self.df.iloc[index]['text']
        label = self.df.iloc[index]['p_sbrt']

        if pd.isna(text):
            text = ' '  ### replaces NaN w/ empty string

        encoding = self.tokenizer.encode_plus(
                                              text,
                                              add_special_tokens = True,
                                              max_length = self.max_length,
                                              return_token_type_ids = False,
                                              padding = 'max_length',
                                              truncation = True,
                                              return_attention_mask = True,
                                              return_tensors = 'pt'
                                              )
        return {
                'input_ids': encoding['input_ids'].flatten(),
                'attention_mask': encoding['attention_mask'].flatten(),
                'label': torch.tensor(label, dtype = torch.long)
                }

# hyperparams

epochs = 2
batch_size = 16
learning_rate = 2e-5
max_length = 512

# define models

models_to_train = {
                   #'BERT': {
                   #         'model_class': BertForSequenceClassification,
                   #         'tokenizer_class': BertTokenizer,
                   #         'pretrained_model_name': 'bert-base-uncased',
                   #         },
                  #'RoBERTa': {
                  #            'model_class': RobertaForSequenceClassification,
                  #            'tokenizer_class': RobertaTokenizer,
                  #            'pretrained_model_name': 'roberta-base',
                  #          },
                  'DistilBERT': {
                                 'model_class': DistilBertForSequenceClassification,
                                 'tokenizer_class': DistilBertTokenizer,
                                 'pretrained_model_name': 'distilbert-base-uncased'
                                 }
                  }

# encode labels

label_encoder = LabelEncoder()
d_adapt['p_sbrt'] = label_encoder.fit_transform(d_adapt['p_sbrt'])

# iterate over models

for model_name, model_info in models_to_train.items():
    print(f'\nTraining {model_name}...')
    print("--------------------------------------------------------------------------------------")

    # initialize tokenizer, model

    tokenizer = model_info['tokenizer_class'].from_pretrained(model_info['pretrained_model_name'])
    model = model_info['model_class'].from_pretrained(
                                                      model_info['pretrained_model_name'],
                                                      num_labels = 3, ### update for true run - maps to tensor dimensions
                                                      )

    # prep dataset

    dataset = CustomDataset(
                            d_adapt,
                            tokenizer,
                            max_length = max_length,
                            )
    dataloader = DataLoader(
                            dataset,
                            batch_size = batch_size,
                            shuffle = True,
                            )

    # config optimizer

    optimizer = AdamW(
                      model.parameters(), ### ensures all model params are trained
                      lr = learning_rate,
                      )

    # training loop

    model.train()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    for epoch in range(epochs):
        print(f'Epoch {epoch+1}/{epochs}')
        for batch in tqdm(dataloader):
            optimizer.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(
                            input_ids = input_ids,
                            attention_mask = attention_mask,
                            labels = labels,
                            )
            loss = outputs.loss
            loss.backward()
            optimizer.step()

    # save

    save_path = f'{models_path}{model_name}_adapted'
    model.save_pretrained(save_path)
    tokenizer.save_pretrained(save_path)

    print(f'{model_name} saved to {save_path}\n')

#### Train and evaluate domain-adapted models: $k$-fold cross validate

In [ ]:
targets_and_class_weights = {
                             'asp': [
                                     0.7657, ### w_neg
                                     1.4410, ### w_pos
                                     ],
                             'dep': [
                                     0.5989,
                                     3.0286,
                                     ],
                             'val': [
                                     0.6876,
                                     1.8327,
                                     ],
                             'prg': [
                                     0.6024,
                                     2.9414,
                                     ],
                             'tgd': [
                                     0.5974,
                                     3.0676,
                                     ],
                             'age': [
                                     0.6303,
                                     2.4188,
                                     ],
                             'race': [
                                      0.506,
                                      42.0441,
                                      ],
                             'dbty': [
                                      0.5064,
                                      39.7083,
                                      ],
                              }

models_path = '/content/drive/MyDrive/Colab/bar_policy_suicidality/outputs/models/'

In [ ]:
targets_and_class_weights = targets_and_class_weights

models = {
          'bert': (
                   BertForSequenceClassification.from_pretrained(f'{models_path}BERT_adapted'),
                   BertTokenizer.from_pretrained(f'{models_path}BERT_adapted'),
                   'bert-base-uncased',
                   ),

          'roberta': (
                      RobertaForSequenceClassification.from_pretrained(f'{models_path}RoBERTa_adapted'),
                      RobertaTokenizer.from_pretrained(f'{models_path}RoBERTa_adapted'),
                      'roberta-base',
                      ),

          'distilbert': (
                         DistilBertForSequenceClassification.from_pretrained(f'{models_path}DistilBERT_adapted'),
                         DistilBertTokenizer.from_pretrained(f'{models_path}DistilBERT_adapted'),
                         'distilbert-base-uncased',
                         )
          }

save_path = '/content/drive/MyDrive/Colab/bar_policy_suicidality/outputs/models/'

cycle = 'adapted'

In [ ]:
%pwd

In [ ]:
%cd ../../outputs/tables

# 'adapted' cycle: train-test loop

results = train_eval_save_bl_models(
                                    d_augmented,
                                    targets_and_class_weights,
                                    models,
                                    save_path,
                                    cycle,
                                    )

**Adapted: viz**

In [ ]:
%cd ../outputs/tables

In [ ]:
%cd /content/drive/MyDrive/Colab/bar_policy_suicidality/outputs/tables

In [ ]:
%%capture

d_v = pd.read_excel('d_adapted_performance.xlsx')
d_v.round({'f1_macro': 4, 'mcc': 4, 'auprc': 4})

In [ ]:
%cd ../figures

performance_barplot(
                    d_v,
                    'adapted_performance',
                    )

### 5. Tune
Builds stratified train-test sets, searches hyperparam space to optimize highest-performing target x pretrained model configs.
***

In [ ]:
%pwd

In [ ]:
%cd ../inputs/data

d_augmented = pd.read_excel('d_augmented.xlsx')

In [ ]:
targets = [
           'asp',
           'dep',
#           'val',
#           'prg',
#           'tgd',
#           'age',
#           'race',
#           'dbty',
           ]

d_augmented = d_augmented[
                          ['text',
                           'aug'] +
                           targets
                           ].copy()

d_augmented.info()
d_augmented.head(3)

In [ ]:
%cd ../../temp

**Target-parsed $\mathcal{d}$<sub>train</sub>($y$): augmented | $\mathcal{d}$<sub>test</sub>($y$): de-augmented**

In [ ]:
target_datasets = iterative_stratified_train_test_split_with_rationales(
                                                               d_augmented,
                                                               targets,
                                                               random_state = 56,
                                                               test_size = 0.2,
                                                               )

In [ ]:
print(target_datasets.keys())

In [ ]:
# drop 'aug' + extract target-wise train/test df

#for target, (d_train, d_test) in target_datasets.items():

        # Drop the 'aug' column from both train and test datasets
#    if 'aug' in d_train.columns:
#        d_train = d_train.drop(columns=['aug'])
#    if 'aug' in d_test.columns:
#        d_test = d_test.drop(columns=['aug'])

        # Update the dictionary with the modified dataframes
d_train_asp, d_test_asp = target_datasets['asp']
d_train_dep, d_test_dep = target_datasets['dep']

In [ ]:
d_train_asp = d_train_asp.drop('aug', axis = 1)
d_test_asp = d_test_asp.drop('aug', axis = 1)

d_train_asp.head(3)
d_test_asp.head(3)

d_train_dep = d_train_dep.drop('aug', axis = 1)
d_test_dep = d_test_dep.drop('aug', axis = 1)

d_train_dep.head(3)
d_test_dep.head(3)

In [ ]:
%cd ../../outputs/tables

# define hyperparameter grid

hyperparameter_grid = {
                       'batch_size': [8, 16],
                       'gradient_accumulation_steps': [1, 2],
                       'learning_rate': [2e-5, 3e-5],
                       'num_epochs': [2, 3],
                       'warmup_steps': [0, 500],
                       'weight_decay': [0.0, 0.3],
                       }

models_path = '/content/drive/MyDrive/Colab/bar_policy_suicidality/outputs/models'

    ### SJS 9/8: no longer using dedicated subdirectories; just do save_path = models_path when calling the Fx...

# define tuning param sets

params = [

    # asp: Best F1 (macro) for asp: 0.2162 achieved by bert - benchmark

          {
           'tokenizer': BertTokenizer.from_pretrained('bert-base-uncased'),
           'model_class': BertForSequenceClassification,
           'pretrained_model_name': 'bert-base-uncased',
           'd_train': d_train_asp,
           'd_test': d_test_asp,
           'target': 'asp',
           'class_weights': torch.tensor([
                                          0.7657, ### w_neg
                                          1.4410, ### w_pos
                                          ], dtype = torch.float),
           'save_path': models_path,
           },

    # dep: Best F1 (macro) for dep: 0.6857 achieved by bert - benchmark

          {
           'tokenizer': BertTokenizer.from_pretrained('bert-base-uncased'),
           'model_class': BertForSequenceClassification,
           'pretrained_model_name': 'bert-base-uncased',
           'd_train': d_train_dep,
           'd_test': d_test_dep,
           'target': 'dep',
           'class_weights': torch.tensor([
                                          0.5989,
                                          3.0286,
                                          ], dtype = torch.float),
           'save_path': models_path,
           },

    # val: Best F1 (macro) for val: 0.6259 achieved by roberta - roberta

#          {
#           'tokenizer': RobertaTokenizer.from_pretrained('roberta-base'),
#           'model_class': RobertaForSequenceClassification,
#           'pretrained_model_name': 'roberta-base',
#           'd_train': d_train_val,
#           'd_test': d_test_val,
#           'target': 'val',
#           'class_weights': torch.tensor([
#                                          0.6876,
#                                          1.8327,
#                                          ], dtype = torch.float),
#           'save_path': models_path,
#           },
     # prg: Best F1 (macro) for prg: 0.6013 achieved by roberta

#          {
#           'tokenizer': RobertaTokenizer.from_pretrained('roberta-base'),
#           'model_class': RobertaForSequenceClassification,
#           'pretrained_model_name': 'roberta-base',
#           'd_train': d_train_prg,
#           'd_test': d_test_prg,
#           'target': 'prg',
#           'class_weights': torch.tensor([
#                                          0.6024,
#                                          2.9414,
#                                          ], dtype = torch.float),
#           'save_path': models_path,
#           },

    # tgd: Best F1 (macro) for tgd: 0.7414 achieved by bert

#          {
#           'tokenizer': BertTokenizer.from_pretrained('bert-base-uncased'),
#           'model_class': BertForSequenceClassification,
#           'pretrained_model_name': 'bert-base-uncased',
#           'd_train': d_train_tgd,
#           'd_test': d_test_tgd,
#           'target': 'tgd',
#           'class_weights': torch.tensor([
#                                          0.5974,
#                                          3.0676,
#                                          ], dtype = torch.float),
#           'save_path': models_path,
#           },

]


# tuning loop

all_tuned_performance = pd.DataFrame()

for p in params:
    #print(f"Inspecting parameter set: {p}")  # inspect for dict format
    d_test, d_tuned_performance = tune_and_optimize_model_hyperparams(
                                                                      tokenizer = p['tokenizer'],
                                                                      model_class = p['model_class'],
                                                                      pretrained_model_name = p['pretrained_model_name'],
                                                                      d_train = p['d_train'],
                                                                      d_test = p['d_test'],
                                                                      target = p['target'],
                                                                      class_weights = p['class_weights'],
                                                                      save_path = p['save_path'],
                                                                      hyperparameter_grid = hyperparameter_grid,
                                                                      )

    all_tuned_performance = pd.concat([all_tuned_performance, d_tuned_performance], ignore_index = True)

print(all_tuned_performance.head())

In [ ]:
print(all_tuned_performance.head())
all_tuned_performance.to_excel('all_tuned_performance.xlsx')

In [ ]:
# ARMHR viz - concat

%cd ../temp/

temp_path = '/content/drive/MyDrive/Colab/bar_policy_suicidality/temp/'

targets = [
           'asp',
           'dep',
           'val',
           'prg',
           'tgd',
           #'age',
           #'race',
           #'dbty',
           ]

d_list = []

for target in targets:
    file_path = os.path.join(temp_path, f'd_tuned_performance_{target}.xlsx')

    d = pd.read_excel(
                       file_path,
                       index_col = [0],
                       )

    d['target'] = target
    d_list.append(d)

d_all_tuned_performance = pd.concat(
                                    d_list,
                                    ignore_index = True,
                                    )

d_all_tuned_performance.info()
d_all_tuned_performance.head(3)

d_all_tuned_performance.to_excel('d_all_tuned_performance.xlsx')

### 6. Infer

In [ ]:
### SJS 9/18: for ARMHR

%cd /content/drive/MyDrive/Colab/bar_policy_suicidality/inputs/data
#%cd inputs/data

d_adapt = pd.read_csv('d_adapt.csv')

d_adapt.info()
d_adapt.head(3)

In [ ]:
d_inference = d_adapt.sample(
                             n = 10000,
                             random_state = 56,
                             ).reset_index(drop = True)

d_inference.info()
d_inference.head(3)

In [ ]:
models_path = '/content/drive/MyDrive/Colab/bar_policy_suicidality/outputs/models/'

# define inference param sets

params = [
          {
           'target': 'asp',
           'model_class': BertForSequenceClassification,
           'tokenizer_class': BertTokenizer,
           'pretrained_model_name': 'bert-base-uncased',
           'model_path': f'{models_path}asp_bert-base-uncased_best_tuned_model.bin',
          },
          {
           'target': 'dep',
           'model_class': BertForSequenceClassification,
           'tokenizer_class': BertTokenizer,
           'pretrained_model_name': 'bert-base-uncased',
           'model_path': f'{models_path}dep_bert-base-uncased_best_tuned_model.bin',
          },
         {
          'target': 'val',
          'model_class': RobertaForSequenceClassification,
          'tokenizer_class': RobertaTokenizer,
          'pretrained_model_name': 'roberta-base',
          'model_path': f'{models_path}val_roberta-base_best_tuned_model.bin',
         },
          {
           'target': 'prg',
          'model_class': RobertaForSequenceClassification,
          'tokenizer_class': RobertaTokenizer,
          'pretrained_model_name': 'roberta-base',
           'model_path': f'{models_path}prg_roberta-base_best_tuned_model.bin',
          },
         {
          'target': 'tgd',
          'model_class': BertForSequenceClassification,
          'tokenizer_class': BertTokenizer,
          'pretrained_model_name': 'bert-base-uncased',
          'model_path': f'{models_path}tgd_bert-base-uncased_best_tuned_model.bin',
         },

    ### SJS 9/8: full gamut TKTK

]

# coerce to str

d_inference['text'] = d_inference['text'].astype(str)
texts = d_inference['text'].tolist()

# inference loop

for p in params:
    target = p['target']

    # load tokenizers, models

    tokenizer = p['tokenizer_class'].from_pretrained(p['pretrained_model_name'])
    model = load_model(
                       p['model_path'],
                       p['model_class'],
                       p['pretrained_model_name'],
                       )

    # infer predictions, probabilities

    predictions, probabilities = predict(
                                         model,
                                         tokenizer,
                                         texts,
                                         )

    d_inference[f'{target}_pred'] = predictions
    d_inference[f'{target}_prob'] = probabilities

    # inspect

d_inference[[
             'asp_pred',
             'dep_pred',
             'val_pred',
             'prg_pred',
             'tgd_pred',
             ]].apply(pd.Series.value_counts)

d_inference.head(3)

In [ ]:
d_inference.to_csv('d_inference_pred_armhr.csv')

### 7. Explain

In [ ]:
%pwd

In [ ]:
%cd inputs/data

### SJS 9/20: Fx TKTK to parsimoniously explain over targets - _or_ multilabel/ensemble...

d_inference = pd.read_csv('d_inference_pred_armhr.csv')
d_inference.drop(
                 'Unnamed: 0',
                 axis = 1,
                 inplace = True,
                 )

d_inference.info()
d_inference.head(3)

#### _$\hat{s}$_<sub>1</sub>: asp clr

**Sort by confidence**

In [ ]:
# extract pos proba

d_inference['asp_prob'] = d_inference['asp_prob'].apply(lambda i: ast.literal_eval(i))
d_inference['asp_pos'] = d_inference['asp_prob'].apply(lambda i: round(i[1], 4))

#d_inference.head()

In [ ]:
# parse by 'asp_pos' > 0.90

pos_index = d_inference[d_inference['asp_pos'] > 0.90]
pos_index

**LIMETextExplainer**

In [ ]:
%%capture

models_path = '/content/drive/MyDrive/Colab/bar_policy_suicidality/outputs/models/'

# load pre-trained tokenizer, config, architecture from Hugging Face

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained(
                                                      'bert-base-uncased',
                                                      num_labels = 2,
                                                      )

# load fine-tuned model weights

asp_path = f'{models_path}asp_bert-base-uncased_best_tuned_model.bin'
model.load_state_dict(torch.load(asp_path, map_location = torch.device('cuda')))

# coerce eval mode

model.eval()

# targets

class_names = [
               '0',
               '1',
               ]

# tokenize Fx

def predict_proba(texts):
    inputs = tokenizer(
                       texts,
                       padding = True,
                       truncation = True,
                       return_tensors = 'pt',
                       max_length = 512,
                       )
    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.softmax(
                              outputs.logits,
                              dim = 1,
                              )
    return probs.cpu().numpy()

# initialize explainer

### SJS 9/27: re 'bow' and 'char_level' params: https://lime-ml.readthedocs.io/en/latest/lime.html

explainer = LimeTextExplainer(
                              class_names = class_names,
                              random_state = 56,
                              #bow = False,
                              #char_level = True,
                              )

# illustrative string - toy ex

#text = '''
#All my life I wished I could be a champion snowboarder, but my injuries as a child ruined that dream.
#Because I am medicated and rely on a walker to navigate only short distances, my athletic career can
#never return. I feel hopeless and stunted and passed by, and like a burden to my family.
#'''

# d_inference selection

pos_instance = 51
text = d_inference.loc[pos_instance, 'text']

# explain

exp = explainer.explain_instance(
                                 text,
                                 predict_proba,
                                 num_features = 8,
                                 num_samples = 1000,
                                 distance_metric = 'cosine',
                                 )


**Display explanation**

In [ ]:
print(pos_instance)
exp.show_in_notebook(text = True)
#exp.save_to_file('lime_explanation.html')

In [ ]:
%cd /content/drive/My Drive/Colab/bar_policy_suicidality/outputs/figures
exp.save_to_file('lime_explanation_asp_51.html')

#### _$\hat{s}$_<sub>2</sub>: dep clr

**Sort by confidence**

In [ ]:
# extract pos proba

#d_inference['dep_prob'] = d_inference['dep_prob'].apply(lambda i: ast.literal_eval(i))
d_inference['dep_pos'] = d_inference['dep_prob'].apply(lambda i: round(i[1], 4))

#d_inference.head()

In [ ]:
# parse by 'asp_pos' > 0.90

pos_index = d_inference[d_inference['dep_pos'] > 0.90]
pos_index

In [ ]:
%%capture

models_path = '/content/drive/MyDrive/Colab/bar_policy_suicidality/outputs/models/'

# load pre-trained tokenizer, config, architecture from Hugging Face

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained(
                                                      'bert-base-uncased',
                                                      num_labels = 2,
                                                      )

# load fine-tuned model weights

dep_path = f'{models_path}dep_bert-base-uncased_best_tuned_model.bin'
model.load_state_dict(torch.load(dep_path, map_location = torch.device('cuda')))

# coerce eval mode

model.eval()

# targets

class_names = [
               '0',
               '1',
               ]

# tokenize Fx

def predict_proba(texts):
    inputs = tokenizer(
                       texts,
                       padding = True,
                       truncation = True,
                       return_tensors = 'pt',
                       max_length = 512,
                       )
    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.softmax(
                              outputs.logits,
                              dim = 1,
                              )
    return probs.cpu().numpy()

# initialize explainer

explainer = LimeTextExplainer(class_names = class_names)

# d_inference selection

pos_instance = 116
text = d_inference.loc[pos_instance, 'text']

# explain

exp = explainer.explain_instance(
                                 text,
                                 predict_proba,
                                 num_features = 8,
                                 num_samples = 1000,
                                 distance_metric = 'cosine',
                                 )


In [ ]:
print(pos_instance)
exp.show_in_notebook(text = True)
#exp.save_to_file('lime_explanation.html')

In [ ]:
%cd /content/drive/My Drive/Colab/bar_policy_suicidality/outputs/figures
exp.save_to_file('lime_explanation_dep_116.html')

#### _$\hat{s}$_<sub>3</sub>: val clr

**Sort by confidence**

In [ ]:
# extract pos proba

#d_inference['val_prob'] = d_inference['val_prob'].apply(lambda i: ast.literal_eval(i))
d_inference['val_pos'] = d_inference['val_prob'].apply(lambda i: round(i[1], 4))

#d_inference.head()

In [ ]:
# parse by 'asp_pos' > 0.90

pos_index = d_inference[d_inference['val_pos'] > 0.90]
pos_index

In [ ]:
%%capture

models_path = '/content/drive/MyDrive/Colab/bar_policy_suicidality/outputs/models/'

# load pre-trained tokenizer, config, architecture from Hugging Face


tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained(
                                                      'roberta-base',
                                                      num_labels = 2,
                                                      )
# load fine-tuned model weights

val_path = f'{models_path}val_roberta-base_best_tuned_model.bin'
model.load_state_dict(torch.load(val_path, map_location = torch.device('cuda')))

# coerce eval mode

model.eval()

# targets

class_names = [
               '0',
               '1',
               ]

# tokenize Fx

def predict_proba(texts):
    inputs = tokenizer(
                       texts,
                       padding = True,
                       truncation = True,
                       return_tensors = 'pt',
                       max_length = 512,
                       )
    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.softmax(
                              outputs.logits,
                              dim = 1,
                              )
    return probs.cpu().numpy()

# initialize explainer

explainer = LimeTextExplainer(class_names = class_names)

# d_inference selection

pos_instance = 19
text = d_inference.loc[pos_instance, 'text']

# explain

exp = explainer.explain_instance(
                                 text,
                                 predict_proba,
                                 num_features = 8,
                                 num_samples = 1000,
                                 distance_metric = 'cosine',
                                 )


In [ ]:
print(pos_instance)
exp.show_in_notebook(text = True)
#exp.save_to_file('lime_explanation.html')

In [ ]:
%cd /content/drive/My Drive/Colab/bar_policy_suicidality/outputs/figures
exp.save_to_file('lime_explanation_val_19.html')

> End of aim_i_train_tune_predict.ipynb